In [ ]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import datajoint as dj
dj.config.load('dj_local_conf.json')
dj.conn()

from workflow.pipeline import scan, imaging, subject, session
from pathlib import Path
import datetime
import numpy as np

In [ ]:
# Make sure specified subject exists within the subject.Subject() table  
# Skip this step if subject has been inserted into subject.Subject already
subj_key = dict(
    subject='test_sub',
    subject_nickname="",
    sex='F',
    subject_birth_date=datetime.date(2022, 1, 1),
    subject_description= "Test subject for Calcium Imaging"
)
subject.Subject.insert1(subj_key)

# Make sure specified session exists in session.Session() and session.SessionDirectory() tables
# Skip these steps if session has been inserted into session tables already
session_key = dict(subject='test_sub', session_id=1,
                   session_datetime='2023-01-07 12:00:00')

session.Session.insert1(session_key)
sdir_key = dict(subject=session_key['subject'], 
                session_id=session_key['session_id'], 
                session_dir='test_sub/session1')
session.SessionDirectory.insert1(sdir_key)

In [ ]:
# Insert ProcessingParamSet
# Can manually specify specific suite2p paramters of interest
params_suite2p = {'look_one_level_down': 0.0,
                  'fast_disk': [],
                  'delete_bin': False,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 10.0,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'save_mat': False,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': False,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'keep_movie_raw': False,
                  'nimg_init': 300,
                  'batch_size': 500,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': True,
                  'block_size': [128, 128],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': False,
                  'diameter': 12,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 5000,
                  'max_iterations': 20,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}

imaging.ProcessingParamSet.insert_new_params(
    processing_method='suite2p', 
    paramset_idx=0, 
    params=params_suite2p,
    paramset_desc='Calcium imaging analysis with Suite2p using default Suite2p parameters')


In [ ]:
# Insert entry into imaging.ProcessingTask
# Calcium imaging pipeline depends on both calcium-imaging-worker and standard-worker
# If workers are active, this insert step will trigger the rest of the processing pipeline
pt_key = dict(
    subject="AEG22",
    session_id=0,
    scan_id=0,
    paramset_idx=0,
    processing_output_dir='Imaging/',
    task_mode="trigger"
)
imaging.ProcessingTask.insert1(pt_key)